In [1]:
!gdown 1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
!gdown 1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1

Downloading...
From: https://drive.google.com/uc?id=1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
To: /content/test_data.csv
100% 565k/565k [00:00<00:00, 74.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1
To: /content/train_data.csv
100% 2.24M/2.24M [00:00<00:00, 126MB/s]


In [2]:
import numpy as np
import pandas as pd
import gensim
import gensim.downloader as api
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler

In [3]:
train_df = pd.read_csv("train_data.csv")
train_df = train_df[train_df['confidence'] >= 0.8].reset_index(drop=True)

In [4]:
test_df = pd.read_csv("test_data.csv")
test_df = test_df[test_df['confidence'] >= 0.8].reset_index(drop=True)

In [5]:
print(train_df.shape, test_df.shape)

(1832, 111) (465, 111)


In [6]:
X_train = train_df.drop(['target'], axis=1)
y_train = train_df['target'].to_numpy()
X_test = test_df.drop(['target'], axis=1)
y_test = test_df['target'].to_numpy()

# Pretrained Word2Vec

In [7]:
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [8]:
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

In [9]:
voc = set(model_w2v.index_to_key)
def document_vector(doc):
    doc = doc.split()
    doc = [word for word in doc if word in voc]
    return np.mean(model_w2v[doc], axis=0)

In [10]:
train_w2v = []
for review in X_train['clean_text']:
    train_w2v.append(document_vector(review))

In [11]:
test_w2v = []
for review in X_test['clean_text']:
    test_w2v.append(document_vector(review))

In [12]:
np.array(train_w2v).shape

(1832, 300)

In [13]:
X_train_w2v = X_train.drop(['clean_text'], axis=1).to_numpy()
X_test_w2v = X_test.drop(['clean_text'], axis=1).to_numpy()

In [14]:
X_train_w2v = np.concatenate((X_train_w2v, train_w2v), axis=1)
X_test_w2v = np.concatenate((X_test_w2v, test_w2v), axis=1)

In [15]:
print(X_train_w2v.shape)
print(X_test_w2v.shape)

(1832, 409)
(465, 409)


# Training and Testing

In [16]:
def training_and_prediction(training_model, X_train, X_test, y_train, y_test):
  model = training_model
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  # print("Accuracy =", accuracy)
  # print("Confusion Matrix =\n", confusion_matrix(y_test, y_pred))
  # print(classification_report(y_test, y_pred))
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [17]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

## Naive Bayes

In [18]:
scaler = MinMaxScaler()
training_and_prediction(MultinomialNB(alpha=2), scaler.fit_transform(X_train_w2v), scaler.fit_transform(X_test_w2v), y_train, y_test)

{'Accuracy': 0.7741935483870968,
 'Precision': 0.7896995708154506,
 'Recall': 0.7666666666666667,
 'F1-Score': 0.7780126849894293}

In [19]:
training_and_prediction(GaussianNB(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7827956989247312,
 'Precision': 0.7662835249042146,
 'Recall': 0.8333333333333334,
 'F1-Score': 0.7984031936127745}

In [22]:
training_and_prediction(BernoulliNB(alpha=10), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7397849462365591,
 'Precision': 0.7117437722419929,
 'Recall': 0.8333333333333334,
 'F1-Score': 0.7677543186180423}

## SVM

In [31]:
training_and_prediction(SVC(kernel='poly', C=3), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7978494623655914,
 'Precision': 0.7786259541984732,
 'Recall': 0.85,
 'F1-Score': 0.8127490039840638}

In [26]:
training_and_prediction(SVC(C=10), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.8021505376344086,
 'Precision': 0.7868217054263565,
 'Recall': 0.8458333333333333,
 'F1-Score': 0.8152610441767068}

## Random Forest

In [32]:
training_and_prediction(RandomForestClassifier(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.8021505376344086,
 'Precision': 0.8057851239669421,
 'Recall': 0.8125,
 'F1-Score': 0.8091286307053941}

## Logistic Regression

In [35]:
training_and_prediction(LogisticRegression(max_iter=10000, solver='saga'), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.8043010752688172,
 'Precision': 0.796812749003984,
 'Recall': 0.8333333333333334,
 'F1-Score': 0.814663951120163}

## Gradient Boosting

In [34]:
training_and_prediction(XGBClassifier(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.8150537634408602,
 'Precision': 0.8208333333333333,
 'Recall': 0.8208333333333333,
 'F1-Score': 0.8208333333333333}

## Deep Learning Techniques with Word2vec

In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

### ANN

In [37]:
def ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate=0):
    model_dense = Sequential()
    model_dense.add(Dense(num_neurons_layer1, activation='relu', input_shape=(X_train_w2v.shape[1],)))
    model_dense.add(Dropout(dropout_rate))
    model_dense.add(Dense(num_neurons_layer2, activation='relu'))
    model_dense.add(Dense(1, activation='sigmoid'))
    model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_dense.fit(X_train_w2v, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_w2v, y_test))
    y_pred_dense = model_dense.predict(X_test_w2v)
    y_pred_dense = (y_pred_dense > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'num_neurons_layer2': num_neurons_layer2, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_dense)]

In [38]:
for num_neurons_layer1 in [256, 128, 64]:
  for num_neurons_layer2 in [128, 64, 32]:
    for dropout_rate in [0.0, 0.1, 0.2, 0.3, 0.4]:
      print(ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate))

15/15 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.0}, {'Accuracy': 0.7849462365591398, 'Precision': 0.7631578947368421, 'Recall': 0.8458333333333333, 'F1-Score': 0.8023715415019763}]
15/15 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.1}, {'Accuracy': 0.8064516129032258, 'Precision': 0.8099173553719008, 'Recall': 0.8166666666666667, 'F1-Score': 0.8132780082987552}]
15/15 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.2}, {'Accuracy': 0.7956989247311828, 'Precision': 0.8085106382978723, 'Recall': 0.7916666666666666, 'F1-Score': 0.7999999999999999}]
15/15 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.3}, {'Accuracy': 0.8172043010752689, 'Precision': 0.7969348659003831, 'Recall': 0.86666666666